# How to run TensorFlow on Apple mac M-series 🍟

The TensorFlow machine learning framework is supposed to automatically detect and prioritise the use of GPUs over CPUs. <br>
However, when using Tensorflow on a M-series (Apple Silicon) mac I have found that TensorFlow does not automatically detect and use your Apple GPU; increasing training time significantly. 

* I have listed the steps below to create an environment which will enable TensorFlow to recognise and use Apple's GPUs on M-series chips.
* I have also included an example comparing Apple's GPU and CPU (using a M1-Pro laptop) in a small TensorFlow ML project. 

### 📦 Environment requirements 

I used Conda to create a new envirnment with python included. Then manually installed the following pip packages. Then manually added other conda packages I needed. I experimented with creating a YAML file with these instructions, however have continued to find issues with package conflicts when automating this process, but this manual method worked.  

**Step-bystep Environment Instructions:**

1. Create a new environment with python.
2. pip install tensorflow-macos
3. pip install tensorflow-metal
4. conda install your other packages such as jupyter, pandas etc...

Tensorflow should now automatically use your Mac M-series GPU if it can locate them.

### 💿 Setup

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [2]:
# Check for GPUs!
print("Num GPUs", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs 1


### 🧪 Example TensorFlow model and data to test

In [6]:
# Create a simple CNN model
model = models.Sequential([
    layers.InputLayer(shape=(128, 128, 1)), 
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Generate random input data (for testing purposes)
x_train = np.random.random((10000, 128, 128, 1))  # 10000 images, 128x128 pixels, grayscale
y_train = np.random.randint(10, size=(10000,))   # Random labels for 10 classes

### 🏃‍♂️‍➡️ Using Apple mac M-series GPUs

In [7]:
%%time

# Train the model for a few epochs (to test GPU usage)
model.fit(x_train, y_train, epochs=2, batch_size=32)

Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.1002 - loss: 2.3855
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.1050 - loss: 2.3025
CPU times: user 11.6 s, sys: 7.35 s, total: 18.9 s
Wall time: 21.3 s


### 🐌 Using Apple mac M-series CPU only - for comparison

In [8]:
%%time 

with tf.device('/CPU:0'):
    model.fit(x_train, y_train, epochs=2, batch_size=32)

Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 138ms/step - accuracy: 0.1061 - loss: 2.3024
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 133ms/step - accuracy: 0.1051 - loss: 2.3023
CPU times: user 7min 19s, sys: 1min 3s, total: 8min 23s
Wall time: 1min 25s
